In [5]:
import os
import numpy as np
import pandas as pd
import statsmodels.api as sm
import warnings


In [6]:
# Ignore warnings
warnings.filterwarnings("ignore")


In [7]:
# Load the data
df = pd.read_csv("./teratogenicity_data.txt")
df = df.set_index("ID")
df.head()

,Teratogenicity,LogP,Aromatic_rings,Molecular_weight,H_bond_acceptors,H_bond_donors,Polar_surface_area,Rotatable_bonds,C,F,...,SPST,TCCX,TCOT,TXCX,XCCX,double_bonds,triple_bonds,number_of_charges,net_charge,number_of_rings
ID,,,,,,,,,,,,,,,,,,,,,
CHEMBL1466,1,2.79,2,336.29,6,2,93.06,2,19,0,...,0,0,0,0,0,4,0,0,0,4
CHEMBL633,1,7.24,3,645.31,3,0,42.68,11,25,0,...,0,0,0,0,0,1,0,0,0,3
CHEMBL1201039,1,1.97,2,431.94,6,2,160.75,5,15,0,...,0,0,0,0,0,5,0,0,0,3
CHEMBL170365,-1,-0.18,0,44.05,1,0,17.07,0,2,0,...,0,0,0,0,0,1,0,0,0,0
CHEMBL223520,1,-7.14,0,484.50,15,11,282.61,6,18,0,...,0,0,0,0,0,0,0,0,0,3


In [8]:
# Obtain X and y matrices
X = df.drop("Teratogenicity", axis = 1).values
print(X)
X = sm.add_constant(X)
print(X)
Y = df.Teratogenicity.values
Y[Y==-1]=0


[[  2.79   2.   336.29 ...   0.     0.     4.  ]
 [  7.24   3.   645.31 ...   0.     0.     3.  ]
 [  1.97   2.   431.94 ...   0.     0.     3.  ]
 ...
 [ -0.75   1.   229.63 ...   0.     0.     1.  ]
 [  3.05   2.   300.74 ...   0.     0.     3.  ]
 [  5.     2.   382.88 ...   0.     0.     4.  ]]
[[ 1.    2.79  2.   ...  0.    0.    4.  ]
 [ 1.    7.24  3.   ...  0.    0.    3.  ]
 [ 1.    1.97  2.   ...  0.    0.    3.  ]
 ...
 [ 1.   -0.75  1.   ...  0.    0.    1.  ]
 [ 1.    3.05  2.   ...  0.    0.    3.  ]
 [ 1.    5.    2.   ...  0.    0.    4.  ]]


# Filter
Filtrar los valors de X que tengan poca varianza. Se puede utilizar la funcion VarianceThreshold de sklearn.feature_selection.
Y transformar los datos utilizando la función StandardScaler de sklearn.preprocessing

In [9]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler

sel = VarianceThreshold(threshold=0.0)
X_train = sel.fit_transform(X)
print(X_train.shape)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled


(521, 220)


array([[ 0.32156876,  0.11601541, -0.2337    , ..., -0.19863632,
        -0.00956074,  0.33175478],
       [ 1.9975069 ,  0.5271993 ,  0.57270875, ..., -0.19863632,
        -0.00956074, -0.06467696],
       [ 0.0127442 ,  0.11601541,  0.01590519, ..., -0.19863632,
        -0.00956074, -0.06467696],
       ...,
       [-1.01164945, -0.29516847, -0.51203654, ..., -0.19863632,
        -0.00956074, -0.85754045],
       [ 0.41948874,  0.11601541, -0.32647015, ..., -0.19863632,
        -0.00956074, -0.06467696],
       [ 1.1538886 ,  0.11601541, -0.11212022, ..., -0.19863632,
        -0.00956074,  0.33175478]], shape=(521, 220))

# Split in train and test
Se puede utilizar train_test_split de la libreria sklearn.model_selection.
Queremos 50% de datos para train y para test.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, Y, test_size=0.5, random_state=42)


# Contruir modelos

Se pide:
- Modelo de regresión logística utilizando lasso
- Modelo de regresión logística utilizando elastic-net

Para cada modelo calcular el AUROC y al accuracy y el número de varialbes utilizadas en el modelo.

In [11]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score




In [12]:
modelo_lasso = LogisticRegressionCV(
    Cs=10,
    cv=5,
    penalty='l1',
    solver='liblinear',       # 'liblinear' o 'saga' para L1
    scoring='accuracy',
    max_iter=1000,
    random_state=42
)
modelo_lasso.fit(X_train, y_train)

y_pred_l1 = modelo_lasso.predict(X_test)
y_pred_proba = modelo_lasso.predict_proba(X_test)
print("====LASSO====")
print('accuracy score: ', accuracy_score(y_test, y_pred_l1))
print('ROC AUC: ',roc_auc_score(y_test, y_pred_proba[:, 1]))

====LASSO====
accuracy score:  0.6628352490421456
ROC AUC:  0.6830741133498291


In [13]:
modelo_elastic = LogisticRegressionCV(
    Cs=10,
    cv=5,
    penalty='elasticnet',
    solver='saga',            # OBLIGATORIO para elasticnet
    l1_ratios=[0.1, 0.3, 0.5, 0.7, 0.9],  # proporcion de L1
    scoring='accuracy',
    max_iter=1000,
    random_state=42
)
modelo_elastic.fit(X_train, y_train)

y_pred_l1 = modelo_elastic.predict(X_test)
y_pred_proba = modelo_elastic.predict_proba(X_test)
print("====ELASTIC-NET====")
print('accuracy score: ', accuracy_score(y_test, y_pred_l1))
print('ROC AUC: ',roc_auc_score(y_test, y_pred_proba[:, 1]))

====ELASTIC-NET====
accuracy score:  0.6666666666666666
ROC AUC:  0.676240131966537


# Construir Modelo Seleccionando mejores k features

- Utilizar la función SelectKBest de sklearn.feature_selection para seleccionar las mejores k features. 
- Con esas k variables realizar la regresión logística con cross-validaton con cv = 5, optimizando los valores de roc_auc.
- Calcular la media de los 5 aurocs obtenidos.
- Seleccionar la k con el valor más alto de media de aurocs y con este valor de k entrenar todo el train y evaluar el modelo con el test.

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score


In [19]:

p=50
FS = np.zeros(p)
k = 1
logreg = LogisticRegression()


In [21]:
for i in range(k,p+1):
    selector = SelectKBest(score_func=f_classif, k=i)
    X_train_kbest = selector.fit_transform(X_train, y_train)
    scores = cross_val_score(logreg, X_train_kbest, y_train, cv=5, scoring='accuracy')
    FS[i-1] = scores.mean()

In [50]:
kbest = int(FS.argmax()) +1

In [51]:
print(FS.max(), FS.argmax())
selector = SelectKBest(f_classif, k=kbest)
selector.fit(X_train, y_train)
cols_idxs = selector.get_support(indices=True)
cols_idxs

X_new_train = X_train[:, cols_idxs]
X_new_train = sm.add_constant(X_new_train)

X_new_test = X_test[:, cols_idxs]
X_new_test = sm.add_constant(X_new_test)

m3 = logreg.fit(X_new_train,y_train)

from sklearn.metrics import roc_auc_score
ypred = m3.predict(X_new_train)
auctrain = roc_auc_score(ypred, y_train)

ypred = m3.predict(X_new_test)
auctest = roc_auc_score(ypred, y_test)


print("AUC_ROC in training error - Feature selection: ", auctrain)
print("AUC_ROC in test error - Feature selection: ", auctest)

ypred = m3.predict(X_new_train)
ACtrain = accuracy_score(ypred, y_train)

ypred = m3.predict(X_new_test)
ACtest = accuracy_score(ypred, y_test)

print("Accuracy in training error - Feature selection: ", ACtrain)
print("Accuracy in test error - Feature selection: ", ACtest)

betas = m3.coef_
n_features = sum(sum(abs(betas)>0))
print("Number of features with Feature Selection: ", n_features)

0.6653846153846154 41
AUC_ROC in training error - Feature selection:  0.760777683854607
AUC_ROC in test error - Feature selection:  0.628664596273292
Accuracy in training error - Feature selection:  0.7615384615384615
Accuracy in test error - Feature selection:  0.6283524904214559
Number of features with Feature Selection:  43
